In [ ]:
# TODO:
# - Check notes on phone
# - Population density
# - Competitors
# - Cost of living impact
# - COVID impact?
# - Footfall?

In [2]:
import pandas as pd
import geopandas as gpd

### Income

In [28]:
# England and Wales
msoa_gdf = gpd.read_file("../data/msoa.geojson")
msoa_gdf = msoa_gdf.drop(columns=["BNG_E", "BNG_N"])

ew_income_df = pd.read_excel("../data/income_ew.xlsx", sheet_name="Total annual income")
ew_income_df.columns = ew_income_df.iloc[3]
ew_income_df = ew_income_df.iloc[4:]

msoa_gdf = msoa_gdf.merge(ew_income_df[["MSOA code", "Total annual income (£)"]], left_on="MSOA21CD", right_on="MSOA code", how="left")
msoa_gdf["Total annual income (£)"] = msoa_gdf["Total annual income (£)"].astype(float)

for _ in range(2):
    missing_income_gdf = msoa_gdf.loc[msoa_gdf["Total annual income (£)"].isna()].copy()
    missing_income_gdf["geometry_with_buffer"] = missing_income_gdf.to_crs("EPSG:27700")["geometry"].buffer(5).to_crs("EPSG:4326")
    not_missing_income_gdf = msoa_gdf.loc[msoa_gdf["Total annual income (£)"].notna()].copy()
    surrounding_msoas_gdf = missing_income_gdf.set_geometry("geometry_with_buffer").sjoin(not_missing_income_gdf)
    estimated_income_df = surrounding_msoas_gdf[["MSOA21NM_left", "Total annual income (£)_right"]].groupby("MSOA21NM_left").agg({"Total annual income (£)_right": "mean"}).reset_index()

    for i, row in estimated_income_df.iterrows():
        msoa_gdf.loc[msoa_gdf["MSOA21NM"] == row["MSOA21NM_left"], "Total annual income (£)"] = row["Total annual income (£)_right"]
msoa_gdf = msoa_gdf.drop(columns=["MSOA code"])
msoa_gdf = msoa_gdf.rename(columns={"Total annual income (£)": "household_income"})

In [23]:
# Scotland
scotland_la_gdf = gpd.read_file("../data/scotland_la.geojson")
scotland_income_df = pd.read_excel("../data/income_s.xlsx", sheet_name="Table 1 (2018)")

In [24]:
scotland_income_df.columns = scotland_income_df.iloc[1]
scotland_income_df = scotland_income_df.iloc[3:]
scotland_income_df = scotland_income_df.rename(columns={"Median": "household_income"})
scotland_income_df = scotland_income_df[["Local Authority", "household_income"]]
scotland_income_df["household_income"] = scotland_income_df["household_income"] * 52

### Population